**ePartogram-GPT using Zephyr 7B Alpha and Pytorch NLP**

------------------------------------------------------

**System Setup**

In [1]:
#install required packages
!pip install -q transformers peft accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain langchain-community langchain-core langchain-huggingface sentence-transformers gradio pypdf SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 25.7 MB/s eta 0:

In [ ]:
# import audio packages
!apt-get update
!apt-get install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio

In [2]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
# import dependencies
import os
import gradio as gr
from google.colab import drive
import csv

import chromadb
#from langchain.llms import HuggingFacePipeline

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [4]:
import speech_recognition as sr
# from pocketsphinx import LiveSpeech

**Define and Train Model**

In [5]:
# specify model huggingface mode name
model_name = "anakin87/zephyr-7b-alpha-sharded"

###### other models:
# "HuggingFaceH4/zephyr-7b-beta"

# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

In [6]:
# function for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [7]:
# load model
model = load_quantized_model(model_name)

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)

# specify stop token ids
stop_token_ids = [0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [8]:
# mount google drive and specify folder path
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/Colab Notebooks/My_PDFs_AIML'

# try:
#     from google.colab import drive
#     drive.mount('/content/gdrive/')
#     folder_path = "/content/gdrive/MyDrive/MyDrive/International_Maternal_Healthcare_Textbook.pdf"
# except:
#     folder_path = "./International_Maternal_Healthcare_Textbook.pdf"

!ls '/content/drive/MyDrive/Colab Notebooks/My_PDFs_AIML'

Mounted at /content/drive
'Clinical Rules Table_4.25.16.pdf'	'Use of an electronic Partograph_Litwin.pdf'
'effectiveness of Epartogram 2019.pdf'	'WHO_Labour Care Guide_English.pdf'
'Epartogram editorial.pdf'		"WHO LCG User's Manual.pdf"


In [9]:
# load pdf files
loader = PyPDFDirectoryLoader(folder_path)
documents = loader.load()
print(documents)

# split the documents in small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) #Change the chunk_size and chunk_overlap as needed
all_splits = text_splitter.split_documents(documents)

# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name = embedding_model_name, model_kwargs = model_kwargs) #TO DO

#embed document chunks
vectordb = Chroma.from_documents(documents = all_splits, embedding = embeddings, persist_directory="chroma_db") #TO DO

# specify the retriever
retriever = vectordb.as_retriever()

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/My_PDFs_AIML/WHO_Labour Care Guide_English.pdf', 'page': 0}, page_content='WHO LABOUR CARE GUIDE\nASSESSMENT\nPLAN\nOxytocin (U/L, drops/min)\nMedicine\nIV fluids\nBaseline  \nFHR <110, ≥160\nFHR  \ndeceleration L\nAmniotic fluid M+++, B\nFetal position P, T\nCaput +++\nMoulding +++\nContractions \nper 10 min ≤2, >5\nDuration of \ncontractions <20, >60\nCervix  \n[Plot X]\n10\n9 ≥ 2h\n8 ≥ 2.5h\n7 ≥ 3h\n6 ≥ 5h\n5 ≥ 6h\nDescent  \n[Plot O]\n5\n4\n3\n2\n1\n0\nBABYWOMANLABOUR PROGRESSMEDICATIONSHARED DECISION-MAKING SUPPORTIVE CARE\nCompanion N\nPain relief N\nOral fluid N\nPosture SP\nPulse <60, ≥120\nSystolic BP <80, ≥140\nDiastolic BP  ≥90\nTemperature ºC <35.0,  \n≥ 37.5\nUrine P++, A++\nALERT\nRisk factorsRuptured membranes [Date                              Time                      ]\nAbbreviations:     Y – Yes, N – No, D – Declined, U – Unknown, SP – Supine, MO – Mobile, E – Early, L – Late, V – Variable, I – Int

<ipython-input-9-861146f92fb9>:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = embedding_model_name, model_kwargs = model_kwargs) #TO DO


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# build huggingface pipeline for using zephyr-7b-alpha
pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2048,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)

# build conversational retrieval chain with memory (rag) using langchain
def create_conversation(query: str, chat_history: list) -> tuple:
    try:

        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            memory=memory
        )

        result = qa_chain({'question': query, 'chat_history': chat_history})
        chat_history.append((query, result['answer']))
        return '', chat_history


    except Exception as e:
        chat_history.append((query, e))
        return '', chat_history

<ipython-input-10-8617d193030b>:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline)


### **Run Chatbot UI**

In [11]:
def save_chat_to_csv(chat_history: list, filename: str = 'chat_history.csv'):
    with open(filename, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for query, response in chat_history:
            writer.writerow([query, response])

In [28]:
# Define Functions for pre-prompt engineering
def pre_prompt_engineering(user_input: str) -> str:
    # Example: Add context or instructions
    context = "You are an expert medical assistant."
    return f"{context} {user_input}"

def create_conversation(query: str, chat_history: list) -> tuple:
    try:
        memory = ConversationBufferMemory(
            memory_key='chat_history', return_messages=False
        )
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm, retriever=retriever, memory=memory
        )

        result = qa_chain({'question': query, 'chat_history': chat_history})
        chat_history.append((query, result['answer']))

        # Save the updated chat history to CSV
        save_chat_to_csv([(query, result['answer'])])

        return '', chat_history
    except Exception as e:
        error_message = str(e)
        chat_history.append((query, error_message))

        # Save the error to CSV as well
        save_chat_to_csv([(query, error_message)])

        return '', chat_history

In [32]:
def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio, language="en-US")
        return text
    except sr.UnknownValueError:
        return "Speech recognition could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results from speech recognition service; {e}"

def process_audio(audio_file):
    if audio_file is not None:
        try:
            recognizer = sr.Recognizer()
            with sr.AudioFile(audio_file) as source:
                audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)
            return text
        except Exception as e:
            return f"Error in speech recognition: {str(e)}"
    return "No audio input received"

In [34]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with your data (Zephyr 7B Alpha)")
    msg = gr.Textbox(label="Enter your message")
    audio_input = gr.Audio(sources=["microphone"], type="filepath")
    clear = gr.ClearButton([msg, chatbot])

    audio_input.change(process_audio, inputs=[audio_input], outputs=[msg])
    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot])
demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0130c6ff859c7570a3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
